## 앙상블 모델과 단일 모델 성능 비교
- 데이터 : sklearn.datasets의 breast_cancer
- 유형 : 지도학습 + 분류
- 방법 : LogisticRegression, DecisionTreeClassifier, SVC
- 학습데이터셋 : 동일한 데이터셋으로 3개의 모델 학습 후 수행

[1] 모듈 로딩 및 데이터 준비

In [5]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

In [6]:
X, y = load_breast_cancer(as_frame=True, return_X_y=True)

In [7]:
X.shape, y.shape

((569, 30), (569,))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=5,
                                                    test_size=0.2,
                                                    stratify=y)

[2] 학습 진행

[2-1] 앙상블 보팅 학습에 사용할 모델 인스턴스 생성

In [9]:
# LogisticRegression 인스턴스
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='liblinear')
lr_model.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [10]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [11]:
# SVC 인스턴스
from sklearn.svm import SVC

svc_model = SVC()
svc_model.fit(X_train,y_train)

SVC()

[2-2] Ensemble 알고리즘 기반 분류

In [13]:
from sklearn.ensemble import VotingClassifier

In [16]:
# 동일 데이터셋으로 병렬 학습 진행할 모델 리스트 선정 및 결과 결정 방법 설정
vt_model = VotingClassifier(estimators=[('lr_model', lr_model),
                                        ('dt_model', dt_model),
                                        ('svc_model', svc_model)],
                            verbose=True
                            )

In [17]:
# 동일 데이터셋을 전달해서 3개의 모델 동시에 학습 진행
vt_model.fit(X_train, y_train)

[Voting] ................. (1 of 3) Processing lr_model, total=   0.0s
[Voting] ................. (2 of 3) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 3) Processing svc_model, total=   0.0s


VotingClassifier(estimators=[('lr_model',
                              LogisticRegression(solver='liblinear')),
                             ('dt_model', DecisionTreeClassifier()),
                             ('svc_model', SVC())],
                 verbose=True)

In [18]:
# 예측하기
vt_model.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1])

In [20]:
# 보팅 인스턴스 내의 학습기들
vt_model.estimators_

[LogisticRegression(solver='liblinear'), DecisionTreeClassifier(), SVC()]

In [22]:
vt_model.named_estimators_

{'lr_model': LogisticRegression(solver='liblinear'),
 'dt_model': DecisionTreeClassifier(),
 'svc_model': SVC()}